In [1]:
import cv2
import matplotlib.pyplot as plt
import json
from google import genai
from google.genai import types
import glob
import os

from IPython.display import display, Markdown
import time
import json

In [2]:
with open("config.json", "r", encoding="utf-8") as f:
   config = json.load(f)
client = genai.Client(api_key=config["gemini_api_key"])
MODEL = config["MODEL"]
prompt_1 = config["prompt_1"]
prompt_2 = config["prompt_2"]

In [3]:
map_resources = {} 

with open("data/mapping.json", "r", encoding="utf-8") as f:
    map_resources["mapping"] = json.load(f)

with open("data/metadata.json", "r", encoding="utf-8") as f:
    map_resources["metadata"] = json.load(f)
    
map_resources["image"] = cv2.cvtColor(cv2.imread("data/map.JPG"), cv2.COLOR_BGR2RGB) 

In [4]:
root_dir = "data/design/"
resources = {}

for apartment in os.listdir(root_dir):
    apt_path = os.path.join(root_dir, apartment)
    if not os.path.isdir(apt_path):
        continue

    resources[apartment] = {}
    for floor in os.listdir(apt_path):
        floor_path = os.path.join(apt_path, floor)
        if not os.path.isdir(floor_path):
            continue

        res = {}
        mapping_file = os.path.join(floor_path, "mapping.json")
        if os.path.exists(mapping_file):
            with open(mapping_file, "r", encoding="utf-8") as f:
                res["mapping"] = json.load(f)

        metadata_file = os.path.join(floor_path, "metadata.json")
        if os.path.exists(metadata_file):
            with open(metadata_file, "r", encoding="utf-8") as f:
                res["metadata"] = json.load(f)

        map_file = os.path.join(floor_path, "design.jpg")
        if os.path.exists(map_file):
            res["image"] = cv2.cvtColor(cv2.imread(map_file), cv2.COLOR_BGR2RGB) 

        resources[apartment][floor] = res

In [5]:
def crop_from_original(resources, code):
    tiles_list = resources["mapping"][code]
    offsets = resources["metadata"]
    min_x, min_y = float("inf"), float("inf")
    max_x, max_y = 0, 0

    for tile_name in tiles_list:
        off = offsets[tile_name]
        x, y, w, h = off["x"], off["y"], off["width"], off["height"]

        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x + w)
        max_y = max(max_y, y + h)

    cropped = resources["image"][min_y:max_y, min_x:max_x]
    return cropped

In [6]:
building_code = "S6.06"
floor_number = 20
apartment_number = int(input("apartment_number: "))

In [7]:
def generate_content(query, image, MODEL, step=1):
    base_config = dict(
        temperature=0.2,
        top_p=0.9,
        top_k=40,
    )

    if step == 1:
        config = types.GenerateContentConfig(**base_config, response_mime_type="application/json",)
    else:
        config = types.GenerateContentConfig(**base_config)
    try: 
        response = client.models.generate_content(
            model=MODEL,
            contents=[image, query],
            config = config
        )
        time.sleep(3)
        return response.text

    except Exception as e:
        print(e)
        return None

In [8]:
map_cropped = crop_from_original(map_resources, building_code)
success, encoded_image = cv2.imencode('.jpg', map_cropped)
img_bytes = encoded_image.tobytes()
map_part  = types.Part.from_bytes(data=img_bytes, mime_type="image/jpeg")

# plt.imshow(map_cropped)
# plt.axis("off")
# plt.show()

In [9]:
floor_ranges = resources[building_code.replace(".", "")].keys()
for i, fr in enumerate(floor_ranges):
    try:
        _, start, end = fr.split("_")
        start, end = int(start), int(end)
        if start <= floor_number <= end:
            floor = fr
            break
    except ValueError:
            continue
    
floor_resources = resources[building_code.replace(".", "")][floor]
design_cropped = crop_from_original(floor_resources, f'CH{apartment_number:02d}')
success, encoded_image = cv2.imencode('.jpg', design_cropped)
img_bytes = encoded_image.tobytes()
design_part  = types.Part.from_bytes(data=img_bytes, mime_type="image/jpeg")

# plt.imshow(design_cropped)
# plt.axis("off")
# plt.show()

### Step 1: Floorplan Analysis Prompt

In [10]:
user_query = f"{prompt_1}\nThông tin căn hộ cần phân tích: CH{apartment_number:02d}"
response_1 = generate_content(user_query, design_part, MODEL)
print(response_1)

{
  "tiếp_giáp": {
    "trên": "trống",
    "dưới": "hành lang",
    "trái": "CH06",
    "phải": "CH08"
  },
  "bố_cục": {
    "cửa_ra_vào": "Cửa ra vào ở cạnh dưới, mở vào từ hành lang.",
    "luồng_chính": "Từ cửa vào, bên trái là khu vực bếp (P.BẾP) và phòng ăn (P.ĂN). Bên phải là phòng khách (P.KHÁCH). Đi thẳng vào trong là WC và phòng ngủ.",
    "không_gian": [
      {
        "tên": "P.Bếp và P.Ăn",
        "vị_trí": "Nằm ngay bên trái cửa ra vào."
      },
      {
        "tên": "P.Khách",
        "vị_trí": "Nằm bên phải cửa ra vào, tiếp giáp với lô gia."
      },
      {
        "tên": "Lô gia",
        "vị_trí": "Ở góc trên bên phải, có lối ra từ phòng khách."
      },
      {
        "tên": "WC",
        "vị_trí": "Nằm ở giữa căn hộ, phía trên khu vực bếp."
      },
      {
        "tên": "P.Ngủ",
        "vị_trí": "Nằm ở phía trên, bên phải WC."
      }
    ]
  },
  "diện_tích": {
    "loại_căn_hộ": "1PN+1",
    "thông_thủy": "42.9m² (TT)",
    "tim_tường": "46.3m² (Tim)"
  

### Step 2: Map Alignment Prompt

In [11]:
user_query = f"""{prompt_2}
Thông tin căn hộ cần phân tích:
- Tòa nhà (Building): {building_code}
- Căn hộ (Apartment): CH{apartment_number:02d}
- Tầng (Floor): {floor}

[MAP_IMAGE]: Đây là ảnh bản đồ tổng quan vị trí block/căn hộ trong khu đô thị.
[FLOORPLAN_LAYOUT]: Đây là JSON mô tả bố cục thiết kế mặt bằng chi tiết của căn hộ CH{apartment_number:02d} trong Block {building_code}: {json.loads(response_1)}
[USER_QUERY]: Lô gia của căn hộ hướng ra khu vực nào?
"""
response_2 = generate_content(user_query, map_part, MODEL, step=2)
display(Markdown(response_2))

Chào anh/chị, tôi đã phân tích chi tiết căn hộ CH07 tại tòa S6.06 và xin gửi đến anh/chị đánh giá như sau:

Lô gia của căn hộ CH07 có hướng chính là hướng **Bắc**, với tầm nhìn hướng thẳng vào khu vực tiện ích và công viên nội khu xanh mát nằm giữa các tòa S6.05 và S10.01. Đây là một hướng rất lý tưởng tại Việt Nam, giúp căn hộ tránh được nắng gắt trực tiếp vào buổi sáng và chiều, giữ cho không gian sống luôn mát mẻ và dễ chịu.

**Phân tích chi tiết về bố cục và trải nghiệm sống:**

*   **Hướng và Bố cục:** Căn hộ có cửa ra vào ở phía Nam, mở từ hành lang chung. Không gian chính bao gồm phòng khách và lô gia được bố trí ở nửa phía Đông của căn hộ, trong khi khu vực bếp và phòng ăn nằm ở nửa phía Tây. Phòng ngủ cũng được đặt ở phía Bắc, có cửa sổ cùng hướng với lô gia, đảm bảo nhận được ánh sáng tự nhiên dịu nhẹ và không khí trong lành từ công viên.
*   **Ưu điểm:**
    1.  **Hướng mát mẻ:** Hướng Bắc giúp giảm đáng kể chi phí làm mát và tạo ra một môi trường sống thoải mái quanh năm.
    2.  **Tầm nhìn nội khu:** Tầm nhìn hướng vào công viên mang lại sự yên tĩnh, trong lành, an toàn và không bị ảnh hưởng bởi tiếng ồn hay khói bụi từ các trục đường lớn.
    3.  **Sự riêng tư:** Do không đối diện trực tiếp với tòa nhà nào quá gần, căn hộ vẫn giữ được sự riêng tư cần thiết.
*   **Nhược điểm:** Vì hai bên tiếp giáp với các căn hộ khác (CH06 phía Tây và CH08 phía Đông), việc đón sáng và thông gió tự nhiên sẽ chủ yếu đến từ mặt thoáng phía Bắc.

**Tư vấn:** Căn hộ S6.06-CH07 là một lựa chọn tuyệt vời cho những ai ưu tiên một không gian sống yên tĩnh, mát mẻ và gần gũi với thiên nhiên. Với tầm nhìn hướng vào lõi cảnh quan trung tâm, đây là sản phẩm lý tưởng cho các gia đình trẻ hoặc những người tìm kiếm một nơi an cư thanh bình.